In [ ]:
import numpy as np
import tensorflow as tf
%matplotlib inline
import os
import function as fn

# GPU
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session( config=config )

data = np.loadtxt('../n_eemd.txt')

label = fn.unit(np.loadtxt('bitcoin_data.txt')[1:]) # 0-1 normalization

depth      = 5 # LSTM depth
hidden_dim = 22 # hidden dimension of LSTM
howmany    = 100 # number of experiments
ratio      = 0.7 # training data set ratio
lr1, lr2   = 0.01, 0.0001 # Starting and ending rate of Exponential decreasing learning rate
batch_size = 512
epochs     = 1700
bandwidth = 0.015 # Parameter in Kernel density Estimation of WES loss function
loss_f = 0 # 0 if MSE loss function, 1 if WES loss function
error_f = fn.error_factor(label, fn.kde_pdf(fn.gaussian_pdf, label, bandwidth), label, 3, loss_f)

predict = [] # Prediction of the native EEMD for MSE loss function
for how in range(howmany):  
    trainX, trainY, testX, testY = fn.split_LSTM(data, label, depth, ratio)
    pred = fn.LSTM(trainX, trainY, testX, testY, depth, hidden_dim, lr1, lr2, batch_size, epochs, error_f)
    predict.append(pred)
    
loss_f = 1
error_f = fn.error_factor(label, fn.kde_pdf(fn.gaussian_pdf, label, bandwidth), label, 3, loss_f)

predict = [] # Prediction of the Native EEMD for WES loss function
for how in range(howmany):  
    trainX, trainY, testX, testY = fn.split_LSTM(data, label, depth, ratio)
    pred = fn.LSTM(trainX, trainY, testX, testY, depth, hidden_dim, lr1, lr2, batch_size, epochs, error_f)
    predict.append(pred)